In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage
import io
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10
plt.rcParams['image.cmap'] = 'gray'

print("="*80)
print("TRANSFORMACIONES DE INTENSIDAD EN IMÁGENES")
print("Tema 6 - Visión Artificial")
print("="*80)

# ==============================================================================
# FUNCIONES AUXILIARES
# ==============================================================================

def calcular_histograma(imagen, nbins=512, normalizado=True):
    """
    Calcula el histograma de una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray
        Imagen de entrada (puede ser float [0,1] o uint8 [0,255])
    nbins : int
        Número de bins del histograma (por defecto 512)
    normalizado : bool
        Si True, normaliza el histograma para que sume 1
    
    Retorna:
    --------
    hist : ndarray
        Valores del histograma
    bins : ndarray
        Centros de los bins
    """
    # Determinar el rango según el tipo de imagen
    if imagen.dtype == np.uint8:
        rango = (0, 255)
    else:
        rango = (0, 1)
    
    hist, bin_edges = np.histogram(imagen.flatten(), bins=nbins, range=rango)
    
    if normalizado:
        hist = hist / imagen.size
    
    # Calcular centros de bins
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    return hist, bins


def mostrar_transformacion(r_vals, s_vals, titulo, xlabel='Intensidad de entrada (r)', 
                          ylabel='Intensidad de salida (s)'):
    """
    Muestra la función de transformación de intensidad.
    
    Parámetros:
    -----------
    r_vals : ndarray
        Valores de intensidad de entrada
    s_vals : ndarray
        Valores de intensidad de salida
    titulo : str
        Título del gráfico
    xlabel : str
        Etiqueta del eje X
    ylabel : str
        Etiqueta del eje Y
    """
    plt.figure(figsize=(8, 8))
    plt.plot(r_vals, s_vals, 'k', linewidth=2.5)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(titulo, fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.axis('square')
    
    # Determinar límites según el rango de datos
    if np.max(r_vals) <= 1:
        plt.xlim([0, 1])
        plt.ylim([0, 1])
    else:
        plt.xlim([0, 255])
        plt.ylim([0, 255])
    
    plt.tight_layout()
    plt.show()


def comparar_imagenes_histogramas(img1, img2, titulo1, titulo2, titulo_general):
    """
    Compara dos imágenes mostrando las imágenes y sus histogramas.
    
    Parámetros:
    -----------
    img1, img2 : ndarray
        Imágenes a comparar
    titulo1, titulo2 : str
        Títulos de cada imagen
    titulo_general : str
        Título general de la figura
    """
    # Calcular histogramas
    hist1, bins1 = calcular_histograma(img1)
    hist2, bins2 = calcular_histograma(img2)
    
    # Crear figura
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Imagen 1
    axes[0, 0].imshow(img1, cmap='gray', vmin=0, vmax=255 if img1.dtype == np.uint8 else 1)
    axes[0, 0].set_title(titulo1, fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    # Imagen 2
    axes[0, 1].imshow(img2, cmap='gray', vmin=0, vmax=255 if img2.dtype == np.uint8 else 1)
    axes[0, 1].set_title(titulo2, fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    
    # Histograma 1
    axes[1, 0].plot(bins1, hist1, 'b', linewidth=1.5)
    axes[1, 0].set_xlabel('Intensidad')
    axes[1, 0].set_ylabel('Frecuencia Normalizada')
    axes[1, 0].set_title(f'Histograma - {titulo1}', fontsize=11)
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_xlim([bins1[0], bins1[-1]])
    
    # Histograma 2
    axes[1, 1].plot(bins2, hist2, 'r', linewidth=1.5)
    axes[1, 1].set_xlabel('Intensidad')
    axes[1, 1].set_ylabel('Frecuencia Normalizada')
    axes[1, 1].set_title(f'Histograma - {titulo2}', fontsize=11)
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].set_xlim([bins2[0], bins2[-1]])
    
    plt.suptitle(titulo_general, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


# ==============================================================================
# CARGAR IMAGEN
# ==============================================================================

print("\n¿Deseas subir tu propia imagen o usar una imagen sintética?")
print("Opciones:")
print("  1. Subir mi propia imagen")
print("  2. Usar imagen sintética generada automáticamente")

try:
    from google.colab import files
    EN_COLAB = True
    print("\n✓ Entorno Google Colab detectado")
except ImportError:
    EN_COLAB = False
    print("\n⚠ No se detectó Google Colab. Se usará imagen sintética por defecto.")

usar_imagen_propia = False

if EN_COLAB:
    opcion = input("\nIngresa tu opción (1 o 2): ").strip()
    
    if opcion == "1":
        print("\n📤 Por favor, sube tu imagen (formatos: JPG, PNG, BMP, etc.)")
        uploaded = files.upload()
        
        if uploaded:
            filename = list(uploaded.keys())[0]
            print(f"\n✓ Imagen cargada: {filename}")
            
            imagen_original = Image.open(io.BytesIO(uploaded[filename]))
            
            if imagen_original.mode != 'L':
                print("  Convirtiendo a escala de grises...")
                imagen_original = imagen_original.convert('L')
            
            imagen_original = np.array(imagen_original)
            
            max_size = 512
            if imagen_original.shape[0] > max_size or imagen_original.shape[1] > max_size:
                print(f"  Redimensionando imagen a máximo {max_size}x{max_size}...")
                ratio = min(max_size / imagen_original.shape[0], max_size / imagen_original.shape[1])
                new_size = (int(imagen_original.shape[1] * ratio), int(imagen_original.shape[0] * ratio))
                img_pil = Image.fromarray(imagen_original)
                img_pil = img_pil.resize(new_size, Image.BILINEAR)
                imagen_original = np.array(img_pil)
            
            usar_imagen_propia = True
            print(f"✓ Imagen lista: {imagen_original.shape}")
        else:
            print("\n⚠ No se subió ninguna imagen. Usando imagen sintética.")

if not usar_imagen_propia:
    print("\n[Generando imagen sintética...]")
    size = 512
    x = np.linspace(0, 4*np.pi, size)
    y = np.linspace(0, 4*np.pi, size)
    X, Y = np.meshgrid(x, y)
    
    # Crear imagen con diferentes regiones de intensidad
    imagen_original = (np.sin(X) * np.cos(Y) + 
                       0.5*np.sin(2*X) * np.cos(2*Y) + 
                       0.3*np.sin(4*X + 4*Y))
    
    imagen_original = ((imagen_original - imagen_original.min()) / 
                       (imagen_original.max() - imagen_original.min()) * 255).astype(np.uint8)
    
    print(f"✓ Imagen sintética creada: {imagen_original.shape}")

print(f"\nInformación de la imagen:")
print(f"  Dimensiones: {imagen_original.shape}")
print(f"  Tipo: {imagen_original.dtype}")
print(f"  Rango: [{imagen_original.min()}, {imagen_original.max()}]")

# Mostrar imagen original
plt.figure(figsize=(8, 8))
plt.imshow(imagen_original, cmap='gray', vmin=0, vmax=255)
titulo = 'Imagen Original Cargada' if usar_imagen_propia else 'Imagen Original (Sintética)'
plt.title(f'{titulo}\n{imagen_original.shape[0]}x{imagen_original.shape[1]} píxeles', 
          fontsize=14, fontweight='bold')
plt.colorbar(label='Intensidad (0-255)')
plt.axis('off')
plt.tight_layout()
plt.show()

# ==============================================================================
# TRANSFORMACIÓN 1: NEGATIVO
# ==============================================================================

print("\n" + "="*80)
print("TRANSFORMACIÓN 1: NEGATIVO (Inversión de Intensidades)")
print("="*80)

print("""
DESCRIPCIÓN:
La transformación negativa invierte los valores de intensidad de la imagen.
Los píxeles oscuros se vuelven claros y viceversa.

FÓRMULA:
    s = (L - 1) - r
    
donde:
    - r: intensidad de entrada [0, L-1]
    - s: intensidad de salida [0, L-1]
    - L: número de niveles de intensidad (256 para imágenes de 8 bits)

APLICACIONES:
    - Visualización de imágenes médicas (rayos X, mamografías)
    - Mejora de detalles en regiones oscuras
    - Inversión de documentos escaneados
""")

def transformacion_negativo(imagen):
    """
    Aplica la transformación negativa a una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray (uint8)
        Imagen de entrada en rango [0, 255]
    
    Retorna:
    --------
    imagen_negativa : ndarray (uint8)
        Imagen con intensidades invertidas
    
    Ejemplo:
    --------
    >>> img_neg = transformacion_negativo(imagen)
    """
    L = 256  # Número de niveles de intensidad
    imagen_negativa = (L - 1) - imagen
    return imagen_negativa.astype(np.uint8)


# Aplicar transformación
print("\n[1] Aplicando transformación negativa...")
img_negativo = transformacion_negativo(imagen_original)

print(f"✓ Transformación completada")
print(f"  Rango original: [{imagen_original.min()}, {imagen_original.max()}]")
print(f"  Rango negativo: [{img_negativo.min()}, {img_negativo.max()}]")

# Mostrar función de transformación
r_vals = np.arange(0, 256)
s_vals = 255 - r_vals
mostrar_transformacion(r_vals, s_vals, 
                      'Función de Transformación Negativa: s = 255 - r')

# Comparar imágenes
comparar_imagenes_histogramas(imagen_original, img_negativo,
                             'Imagen Original', 'Imagen Negativa',
                             'Transformación Negativa')

print("✓ Transformación 1 completada\n")

# ==============================================================================
# TRANSFORMACIÓN 2: LOGARÍTMICA
# ==============================================================================

print("\n" + "="*80)
print("TRANSFORMACIÓN 2: LOGARÍTMICA (Expansión de Regiones Oscuras)")
print("="*80)

print("""
DESCRIPCIÓN:
La transformación logarítmica expande los valores de intensidad bajos (oscuros)
y comprime los valores altos (claros). Es útil para visualizar detalles en
regiones oscuras de la imagen.

FÓRMULA:
    s = c * log(1 + r)
    
donde:
    - r: intensidad de entrada [0, 255]
    - s: intensidad de salida
    - c: constante de escala (por defecto c=1)
    - log: logaritmo natural

CARACTERÍSTICAS:
    - Expande el rango dinámico de píxeles oscuros
    - Comprime el rango dinámico de píxeles claros
    - Útil para imágenes con rango dinámico muy amplio

APLICACIONES:
    - Visualización de espectros de Fourier
    - Imágenes con iluminación no uniforme
    - Imágenes astronómicas
    - Compresión de rango dinámico
""")

def transformacion_logaritmica(imagen, c=1.0):
    """
    Aplica la transformación logarítmica a una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray (uint8)
        Imagen de entrada en rango [0, 255]
    c : float
        Constante de escala (por defecto 1.0)
    
    Retorna:
    --------
    imagen_log : ndarray (uint8)
        Imagen transformada logarítmicamente
    
    Proceso:
    --------
    1. Convertir imagen a float
    2. Aplicar transformación: s = c * log(1 + r)
    3. Normalizar resultado al rango [0, 255]
    4. Convertir a uint8
    
    Ejemplo:
    --------
    >>> img_log = transformacion_logaritmica(imagen, c=1.0)
    """
    # Convertir a float para evitar overflow
    imagen_float = imagen.astype(float)
    
    # Aplicar transformación logarítmica
    imagen_log = c * np.log(1 + imagen_float)
    
    # Normalizar al rango [0, 255]
    imagen_log = (imagen_log - imagen_log.min()) / (imagen_log.max() - imagen_log.min()) * 255
    
    return imagen_log.astype(np.uint8)


# Aplicar transformación
print("\n[2] Aplicando transformación logarítmica...")
c = 1.0
img_log = transformacion_logaritmica(imagen_original, c=c)

print(f"✓ Transformación completada con c={c}")
print(f"  Rango original: [{imagen_original.min()}, {imagen_original.max()}]")
print(f"  Rango logarítmico: [{img_log.min()}, {img_log.max()}]")

# Mostrar función de transformación
r_vals = np.linspace(0, 255, 256)
s_vals = c * np.log(1 + r_vals)
# Normalizar para visualización
s_vals = (s_vals - s_vals.min()) / (s_vals.max() - s_vals.min()) * 255

mostrar_transformacion(r_vals, s_vals, 
                      f'Función de Transformación Logarítmica: s = {c} * log(1 + r)')

# Comparar imágenes
comparar_imagenes_histogramas(imagen_original, img_log,
                             'Imagen Original', 'Imagen Logarítmica',
                             'Transformación Logarítmica')

print("✓ Transformación 2 completada\n")

# ==============================================================================
# TRANSFORMACIÓN 3: CORRECCIÓN GAMMA
# ==============================================================================

print("\n" + "="*80)
print("TRANSFORMACIÓN 3: CORRECCIÓN GAMMA (Control No Lineal de Brillo)")
print("="*80)

print("""
DESCRIPCIÓN:
La corrección gamma es una transformación no lineal que ajusta el brillo
de la imagen mediante una función potencial. Permite controlar de forma
selectiva las regiones oscuras, medias o claras.

FÓRMULA:
    s = c * r^γ
    
donde:
    - r: intensidad de entrada normalizada [0, 1]
    - s: intensidad de salida normalizada [0, 1]
    - γ (gamma): parámetro de corrección
    - c: constante de escala (normalmente c=1)

COMPORTAMIENTO SEGÚN γ:
    - γ < 1: Expande regiones oscuras (aclara la imagen)
    - γ = 1: Sin cambio (transformación identidad)
    - γ > 1: Expande regiones claras (oscurece la imagen)

VALORES TÍPICOS:
    - γ = 0.3 - 0.5: Corrección fuerte (imágenes muy oscuras)
    - γ = 0.5 - 0.8: Corrección moderada
    - γ = 1.5 - 2.5: Oscurecimiento (imágenes sobreexpuestas)

APLICACIONES:
    - Corrección de gamma de monitores y cámaras
    - Ajuste de brillo en imágenes médicas
    - Compensación de iluminación no uniforme
    - Mejora de contraste en regiones específicas
""")

def transformacion_gamma(imagen, gamma=1.0, c=1.0):
    """
    Aplica la corrección gamma a una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray (uint8)
        Imagen de entrada en rango [0, 255]
    gamma : float
        Parámetro gamma (γ)
        - γ < 1: aclara la imagen
        - γ = 1: sin cambio
        - γ > 1: oscurece la imagen
    c : float
        Constante de escala (por defecto 1.0)
    
    Retorna:
    --------
    imagen_gamma : ndarray (uint8)
        Imagen con corrección gamma aplicada
    
    Proceso:
    --------
    1. Normalizar imagen al rango [0, 1]
    2. Aplicar transformación: s = c * r^γ
    3. Escalar resultado al rango [0, 255]
    4. Convertir a uint8
    
    Ejemplo:
    --------
    >>> # Aclarar imagen oscura
    >>> img_gamma = transformacion_gamma(imagen, gamma=0.5)
    >>> 
    >>> # Oscurecer imagen clara
    >>> img_gamma = transformacion_gamma(imagen, gamma=2.0)
    """
    # Normalizar al rango [0, 1]
    imagen_norm = imagen.astype(float) / 255.0
    
    # Aplicar corrección gamma
    imagen_gamma = c * np.power(imagen_norm, gamma)
    
    # Escalar al rango [0, 255]
    imagen_gamma = (imagen_gamma * 255).clip(0, 255)
    
    return imagen_gamma.astype(np.uint8)


# Aplicar transformación con diferentes valores de gamma
print("\n[3] Aplicando corrección gamma con diferentes valores...")

gammas = [0.3, 0.5, 1.0, 1.5, 2.5]
imagenes_gamma = {}

for g in gammas:
    imagenes_gamma[g] = transformacion_gamma(imagen_original, gamma=g)
    print(f"  γ = {g}: rango [{imagenes_gamma[g].min()}, {imagenes_gamma[g].max()}]")

print("✓ Transformaciones completadas")

# Mostrar funciones de transformación
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

r_vals = np.linspace(0, 1, 256)
for g in [0.3, 0.5, 1.0, 1.5, 2.5]:
    s_vals = np.power(r_vals, g)
    label = f'γ = {g}'
    if g == 1.0:
        axes[0].plot(r_vals, s_vals, 'k--', linewidth=2.5, label=label)
    else:
        axes[0].plot(r_vals, s_vals, linewidth=2, label=label)

axes[0].set_xlabel('Intensidad de entrada (r)', fontsize=11)
axes[0].set_ylabel('Intensidad de salida (s)', fontsize=11)
axes[0].set_title('Funciones de Corrección Gamma', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axis('square')
axes[0].set_xlim([0, 1])
axes[0].set_ylim([0, 1])

# Comparación de resultados
axes[1].imshow(imagen_original, cmap='gray')
axes[1].set_title('Imagen Original', fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Mostrar comparación de imágenes con diferentes gammas
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(imagen_original, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[0, 0].axis('off')

for idx, g in enumerate([0.3, 0.5, 1.5, 2.5]):
    row = (idx + 1) // 3
    col = (idx + 1) % 3
    axes[row, col].imshow(imagenes_gamma[g], cmap='gray', vmin=0, vmax=255)
    axes[row, col].set_title(f'γ = {g}', fontsize=11, fontweight='bold')
    axes[row, col].axis('off')

# Ocultar último subplot
axes[1, 2].axis('off')

plt.suptitle('Corrección Gamma con Diferentes Valores', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Comparación detallada con gamma óptimo
gamma_optimo = 0.5
comparar_imagenes_histogramas(imagen_original, imagenes_gamma[gamma_optimo],
                             'Imagen Original', f'Corrección Gamma (γ={gamma_optimo})',
                             'Corrección Gamma')

print("✓ Transformación 3 completada\n")

# ==============================================================================
# TRANSFORMACIÓN 4: ESTIRAMIENTO LINEAL (NORMALIZACIÓN)
# ==============================================================================

print("\n" + "="*80)
print("TRANSFORMACIÓN 4: ESTIRAMIENTO LINEAL (Normalización de Contraste)")
print("="*80)

print("""
DESCRIPCIÓN:
El estiramiento lineal (o normalización) expande el rango de intensidades
de la imagen para que ocupe todo el rango disponible [0, 255]. Mejora el
contraste cuando la imagen tiene un rango dinámico limitado.

FÓRMULA:
    s = (r - r_min) / (r_max - r_min) * (s_max - s_min) + s_min
    
donde:
    - r: intensidad de entrada
    - r_min, r_max: mínimo y máximo de la imagen de entrada
    - s_min, s_max: rango de salida deseado (típicamente [0, 255])
    - s: intensidad de salida

CARACTERÍSTICAS:
    - Transformación lineal (preserva relaciones de intensidad)
    - Maximiza el uso del rango dinámico disponible
    - No altera la distribución relativa de intensidades

APLICACIONES:
    - Mejora de contraste en imágenes con bajo contraste
    - Preprocesamiento para análisis de imagen
    - Normalización antes de operaciones de procesamiento
    - Visualización de imágenes científicas
""")

def estiramiento_lineal(imagen, s_min=0, s_max=255):
    """
    Aplica estiramiento lineal (normalización) a una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray (uint8)
        Imagen de entrada
    s_min : int
        Valor mínimo de salida (por defecto 0)
    s_max : int
        Valor máximo de salida (por defecto 255)
    
    Retorna:
    --------
    imagen_estirada : ndarray (uint8)
        Imagen con contraste estirado
    
    Proceso:
    --------
    1. Encontrar r_min y r_max de la imagen
    2. Aplicar transformación lineal
    3. Escalar al rango [s_min, s_max]
    
    Ejemplo:
    --------
    >>> img_estirada = estiramiento_lineal(imagen)
    """
    # Convertir a float para evitar overflow
    imagen_float = imagen.astype(float)
    
    # Encontrar mínimo y máximo
    r_min = imagen_float.min()
    r_max = imagen_float.max()
    
    # Evitar división por cero
    if r_max == r_min:
        return imagen
    
    # Aplicar estiramiento lineal
    imagen_estirada = (imagen_float - r_min) / (r_max - r_min) * (s_max - s_min) + s_min
    
    return imagen_estirada.astype(np.uint8)


# Simular imagen con bajo contraste
print("\n[4] Simulando imagen con bajo contraste...")
img_bajo_contraste = (imagen_original.astype(float) * 0.3).astype(np.uint8)

print(f"  Rango original: [{imagen_original.min()}, {imagen_original.max()}]")
print(f"  Rango atenuado: [{img_bajo_contraste.min()}, {img_bajo_contraste.max()}]")

# Aplicar estiramiento lineal
print("\n[4] Aplicando estiramiento lineal...")
img_estirada = estiramiento_lineal(img_bajo_contraste)

print(f"✓ Transformación completada")
print(f"  Rango antes: [{img_bajo_contraste.min()}, {img_bajo_contraste.max()}]")
print(f"  Rango después: [{img_estirada.min()}, {img_estirada.max()}]")

# Mostrar función de transformación
r_min = img_bajo_contraste.min()
r_max = img_bajo_contraste.max()
r_vals = np.linspace(r_min, r_max, 100)
s_vals = (r_vals - r_min) / (r_max - r_min) * 255

plt.figure(figsize=(8, 8))
plt.plot(r_vals, s_vals, 'k', linewidth=2.5)
plt.xlabel('Intensidad de entrada (r)', fontsize=12)
plt.ylabel('Intensidad de salida (s)', fontsize=12)
plt.title('Función de Estiramiento Lineal', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])
plt.axis('square')
plt.tight_layout()
plt.show()

# Comparar imágenes
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Fila 1: Imágenes
axes[0, 0].imshow(imagen_original, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original', fontsize=11, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_bajo_contraste, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Bajo Contraste (30%)', fontsize=11, fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_estirada, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Estiramiento Lineal', fontsize=11, fontweight='bold')
axes[0, 2].axis('off')

# Fila 2: Histogramas
hist_orig, bins_orig = calcular_histograma(imagen_original)
hist_bajo, bins_bajo = calcular_histograma(img_bajo_contraste)
hist_est, bins_est = calcular_histograma(img_estirada)

axes[1, 0].plot(bins_orig, hist_orig, 'k', linewidth=1.5)
axes[1, 0].set_title('Histograma Original', fontsize=10)
axes[1, 0].set_xlim([0, 255])
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(bins_bajo, hist_bajo, 'b', linewidth=1.5)
axes[1, 1].set_title('Histograma Bajo Contraste', fontsize=10)
axes[1, 1].set_xlim([0, 255])
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].plot(bins_est, hist_est, 'r', linewidth=1.5)
axes[1, 2].set_title('Histograma Estirado', fontsize=10)
axes[1, 2].set_xlim([0, 255])
axes[1, 2].grid(True, alpha=0.3)

plt.suptitle('Estiramiento Lineal de Contraste', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Transformación 4 completada\n")

# ==============================================================================
# TRANSFORMACIÓN 5: ESTIRAMIENTO NO LINEAL
# ==============================================================================

print("\n" + "="*80)
print("TRANSFORMACIÓN 5: ESTIRAMIENTO NO LINEAL (Mejora Selectiva de Contraste)")
print("="*80)

print("""
DESCRIPCIÓN:
El estiramiento no lineal mejora el contraste de forma selectiva usando
una función sigmoidal. Permite controlar qué regiones de intensidad se
expanden más.

FÓRMULA:
    s = a / (1 + (K / r)^e)
    
donde:
    - r: intensidad de entrada normalizada [0, 1]
    - a: factor de escala (típicamente a=1)
    - K: parámetro de forma (controla el punto de inflexión)
    - e: exponente (controla la pendiente de la curva)
    - s: intensidad de salida

PARÁMETROS:
    - K pequeño (0.1-0.2): Expande más las regiones oscuras
    - K grande (0.5-0.8): Expande más las regiones claras
    - e pequeño (1-2): Transición suave
    - e grande (5-10): Transición abrupta

CARACTERÍSTICAS:
    - Transformación no lineal tipo sigmoide
    - Control fino sobre regiones específicas de intensidad
    - Preserva mejor los detalles que el estiramiento lineal

APLICACIONES:
    - Mejora de contraste adaptativa
    - Realce de detalles en regiones específicas
    - Procesamiento de imágenes médicas
    - Mejora de imágenes con iluminación no uniforme
""")

def estiramiento_no_lineal(imagen, a=1.0, K=0.15, e=1.5):
    """
    Aplica estiramiento no lineal de contraste a una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray (uint8)
        Imagen de entrada
    a : float
        Factor de escala (por defecto 1.0)
    K : float
        Parámetro de forma (por defecto 0.15)
        - Valores pequeños (0.1-0.2): expande regiones oscuras
        - Valores grandes (0.5-0.8): expande regiones claras
    e : float
        Exponente (por defecto 1.5)
        - Valores pequeños (1-2): transición suave
        - Valores grandes (5-10): transición abrupta
    
    Retorna:
    --------
    imagen_estirada : ndarray (uint8)
        Imagen con estiramiento no lineal aplicado
    
    Proceso:
    --------
    1. Normalizar imagen al rango [0, 1]
    2. Aplicar transformación: s = a / (1 + (K/r)^e)
    3. Normalizar resultado al rango [0, 255]
    
    Ejemplo:
    --------
    >>> # Expandir regiones oscuras
    >>> img_nl = estiramiento_no_lineal(imagen, K=0.15, e=1.5)
    """
    # Normalizar al rango [0, 1]
    imagen_norm = imagen.astype(float) / 255.0
    
    # Evitar división por cero
    imagen_norm = np.maximum(imagen_norm, 1e-10)
    
    # Aplicar estiramiento no lineal
    imagen_estirada = a / (1 + np.power(K / imagen_norm, e))
    
    # Normalizar al rango [0, 255]
    imagen_estirada = (imagen_estirada - imagen_estirada.min()) / \
                      (imagen_estirada.max() - imagen_estirada.min()) * 255
    
    return imagen_estirada.astype(np.uint8)


# Aplicar transformación
print("\n[5] Aplicando estiramiento no lineal...")
a, K, e = 1.0, 0.15, 1.5
img_no_lineal = estiramiento_no_lineal(imagen_original, a=a, K=K, e=e)

print(f"✓ Transformación completada con a={a}, K={K}, e={e}")
print(f"  Rango original: [{imagen_original.min()}, {imagen_original.max()}]")
print(f"  Rango estirado: [{img_no_lineal.min()}, {img_no_lineal.max()}]")

# Mostrar función de transformación
r_vals = np.linspace(0.01, 1, 256)
s_vals = a / (1 + np.power(K / r_vals, e))
# Normalizar para visualización
s_vals = (s_vals - s_vals.min()) / (s_vals.max() - s_vals.min())

plt.figure(figsize=(8, 8))
plt.plot(r_vals, s_vals, 'k', linewidth=2.5)
plt.xlabel('Intensidad de entrada (r)', fontsize=12)
plt.ylabel('Intensidad de salida (s)', fontsize=12)
plt.title(f'Función de Estiramiento No Lineal\na={a}, K={K}, e={e}', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.axis('square')
plt.tight_layout()
plt.show()

# Comparar con estiramiento lineal
comparar_imagenes_histogramas(img_estirada, img_no_lineal,
                             'Estiramiento Lineal', 'Estiramiento No Lineal',
                             'Comparación: Lineal vs No Lineal')

print("✓ Transformación 5 completada\n")

# ==============================================================================
# TRANSFORMACIÓN 6: ECUALIZACIÓN DE HISTOGRAMA
# ==============================================================================

print("\n" + "="*80)
print("TRANSFORMACIÓN 6: ECUALIZACIÓN DE HISTOGRAMA (Distribución Uniforme)")
print("="*80)

print("""
DESCRIPCIÓN:
La ecualización de histograma redistribuye las intensidades de la imagen
para que el histograma resultante sea aproximadamente uniforme. Maximiza
el contraste global de la imagen.

PRINCIPIO:
La ecualización utiliza la función de distribución acumulativa (CDF) del
histograma para transformar las intensidades. El objetivo es que todas las
intensidades tengan aproximadamente la misma frecuencia.

FÓRMULA:
    s = (L - 1) * CDF(r)
    
donde:
    - r: intensidad de entrada
    - CDF(r): función de distribución acumulativa
    - L: número de niveles de intensidad (256)
    - s: intensidad de salida

CARACTERÍSTICAS:
    - Maximiza el contraste global
    - Distribución uniforme del histograma
    - Transformación automática (no requiere parámetros)
    - No reversible

VENTAJAS:
    - Mejora automática de contraste
    - Efectiva para imágenes con bajo contraste
    - Realza detalles en toda la imagen

DESVENTAJAS:
    - Puede sobre-amplificar el ruido
    - Puede generar artefactos en imágenes con histogramas bimodales
    - No siempre produce resultados visualmente agradables

APLICACIONES:
    - Mejora de imágenes médicas
    - Procesamiento de imágenes satelitales
    - Mejora de contraste en fotografía
    - Preprocesamiento para reconocimiento de patrones
""")

def ecualizacion_histograma(imagen):
    """
    Aplica ecualización de histograma a una imagen.
    
    Parámetros:
    -----------
    imagen : ndarray (uint8)
        Imagen de entrada
    
    Retorna:
    --------
    imagen_ecualizada : ndarray (uint8)
        Imagen con histograma ecualizado
    transformacion : ndarray
        Función de transformación aplicada (CDF escalada)
    
    Proceso:
    --------
    1. Calcular histograma de la imagen
    2. Calcular CDF (función de distribución acumulativa)
    3. Normalizar CDF al rango [0, 255]
    4. Mapear intensidades usando la CDF
    
    Ejemplo:
    --------
    >>> img_eq, T = ecualizacion_histograma(imagen)
    """
    # Calcular histograma
    hist, _ = np.histogram(imagen.flatten(), bins=256, range=(0, 256))
    
    # Calcular CDF (función de distribución acumulativa)
    cdf = hist.cumsum()
    
    # Normalizar CDF al rango [0, 255]
    cdf_normalized = (cdf - cdf.min()) / (cdf.max() - cdf.min()) * 255
    
    # Mapear intensidades usando la CDF
    imagen_ecualizada = cdf_normalized[imagen]
    
    return imagen_ecualizada.astype(np.uint8), cdf_normalized


# Aplicar ecualización
print("\n[6] Aplicando ecualización de histograma...")
img_ecualizada, transformacion = ecualizacion_histograma(imagen_original)

print(f"✓ Transformación completada")
print(f"  Rango original: [{imagen_original.min()}, {imagen_original.max()}]")
print(f"  Rango ecualizado: [{img_ecualizada.min()}, {img_ecualizada.max()}]")

# Mostrar función de transformación
r_vals = np.arange(0, 256)
s_vals = transformacion

mostrar_transformacion(r_vals, s_vals, 
                      'Función de Transformación de Ecualización de Histograma')

# Comparar imágenes
comparar_imagenes_histogramas(imagen_original, img_ecualizada,
                             'Imagen Original', 'Imagen Ecualizada',
                             'Ecualización de Histograma')

# Comparación adicional con imagen de bajo contraste
print("\n[6] Comparando ecualización en imagen de bajo contraste...")
img_eq_bajo = ecualizacion_histograma(img_bajo_contraste)[0]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img_bajo_contraste, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Bajo Contraste', fontsize=11, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_estirada, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Estiramiento Lineal', fontsize=11, fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_eq_bajo, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Ecualización', fontsize=11, fontweight='bold')
axes[0, 2].axis('off')

# Histogramas
hist_bajo, bins_bajo = calcular_histograma(img_bajo_contraste)
hist_est, bins_est = calcular_histograma(img_estirada)
hist_eq, bins_eq = calcular_histograma(img_eq_bajo)

axes[1, 0].plot(bins_bajo, hist_bajo, 'b', linewidth=1.5)
axes[1, 0].set_title('Histograma Bajo Contraste', fontsize=10)
axes[1, 0].set_xlim([0, 255])
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(bins_est, hist_est, 'g', linewidth=1.5)
axes[1, 1].set_title('Histograma Estirado', fontsize=10)
axes[1, 1].set_xlim([0, 255])
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].plot(bins_eq, hist_eq, 'r', linewidth=1.5)
axes[1, 2].set_title('Histograma Ecualizado', fontsize=10)
axes[1, 2].set_xlim([0, 255])
axes[1, 2].grid(True, alpha=0.3)

plt.suptitle('Comparación de Métodos de Mejora de Contraste', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Transformación 6 completada\n")

# ==============================================================================
# RESUMEN FINAL Y COMPARACIÓN GLOBAL
# ==============================================================================

print("\n" + "="*80)
print("RESUMEN DE TODAS LAS TRANSFORMACIONES")
print("="*80)

# Crear figura comparativa con todas las transformaciones
fig, axes = plt.subplots(3, 3, figsize=(16, 16))

transformaciones = [
    (imagen_original, 'Original'),
    (img_negativo, 'Negativo'),
    (img_log, 'Logarítmica'),
    (imagenes_gamma[0.5], 'Gamma (γ=0.5)'),
    (img_bajo_contraste, 'Bajo Contraste'),
    (img_estirada, 'Estiramiento Lineal'),
    (img_no_lineal, 'Estiramiento No Lineal'),
    (img_ecualizada, 'Ecualización'),
]

for idx, (img, titulo) in enumerate(transformaciones):
    row = idx // 3
    col = idx % 3
    axes[row, col].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[row, col].set_title(titulo, fontsize=12, fontweight='bold')
    axes[row, col].axis('off')

# Ocultar último subplot
axes[2, 2].axis('off')

plt.suptitle('Comparación de Todas las Transformaciones de Intensidad', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("""
RESUMEN DE TRANSFORMACIONES:

1. NEGATIVO
   - Fórmula: s = 255 - r
   - Uso: Inversión de intensidades, imágenes médicas

2. LOGARÍTMICA
   - Fórmula: s = c * log(1 + r)
   - Uso: Expansión de regiones oscuras, compresión de rango dinámico

3. CORRECCIÓN GAMMA
   - Fórmula: s = c * r^γ
   - Uso: Control no lineal de brillo, corrección de monitores
   - γ < 1: aclara, γ > 1: oscurece

4. ESTIRAMIENTO LINEAL
   - Fórmula: s = (r - r_min) / (r_max - r_min) * 255
   - Uso: Normalización, mejora de contraste lineal

5. ESTIRAMIENTO NO LINEAL
   - Fórmula: s = a / (1 + (K/r)^e)
   - Uso: Mejora selectiva de contraste, control fino

6. ECUALIZACIÓN DE HISTOGRAMA
   - Fórmula: s = 255 * CDF(r)
   - Uso: Maximización de contraste global, distribución uniforme

TODAS LAS TRANSFORMACIONES COMPLETADAS EXITOSAMENTE
""")

print("="*80)
print("FIN DEL SCRIPT")
print("="*80)